In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json, pickle
import os
from tensorflow.keras import layers

In [2]:
nasnet = tf.keras.models.load_model('/scratch/st-ipor-1/mkhademi/image_data/NASNetlarge.h5')
img_rows, img_cols, channels = 56, 56, 3
data_dir = '/arc/project/st-ipor-1/mkhademi/BOLD5000_2.0/'
output_dir = '/scratch/st-ipor-1/mkhademi/'
subjs = ['CSI1', 'CSI2', 'CSI3', 'CSI4']
sub = subjs[0]  
bold_imgnames = []
f = open(data_dir + sub + '_imgnames.txt', 'r')
for imgname in f:
    bold_imgnames.append(imgname[:-1])
f.close()

2022-05-06 07:07:12.389575: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-06 07:07:12.434409: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla V100-SXM2-16GB major: 7 minor: 0 memoryClockRate(GHz): 1.53
pciBusID: 0000:1a:00.0
2022-05-06 07:07:12.437446: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2022-05-06 07:07:12.441485: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2022-05-06 07:07:12.445180: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2022-05-06 07:07:12.447547: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10.0
2022-05-06 07:07:12.451893: I tensorflow/stream_executor/platfo

In [3]:
batch_size = 10
pre_dataset = tf.data.TFRecordDataset(
    filenames = [output_dir + 'image_data/' + sub + 'bold5000_coco.tfrecords']).batch(batch_size)
def pre_tf_parse(eg):
    example = tf.io.parse_example(
        eg, {
            'x': tf.io.FixedLenFeature(shape=(71, 89, 72, 1), dtype=tf.float32),
            'yhat': tf.io.FixedLenFeature(shape=(1000), dtype=tf.float32),
        })
    return example['x'], example['yhat']
pre_dataset = pre_dataset.map(pre_tf_parse)
pre_model = tf.keras.Sequential([
    tf.keras.Input(shape=(71, 89, 72, 1)),
    tf.keras.layers.Conv3D(1, 3, activation='relu', padding='same', name='conv1'),
    #tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
    tf.keras.layers.Conv3D(2, 3, activation='relu', padding='same', name='conv2'),
    tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
    tf.keras.layers.Conv3D(4, 3, padding='same', name='conv3'),
    tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(rate=0.4),
    # tf.keras.layers.Dense(100, activation='sigmoid'),
    tf.keras.layers.Dense(1000),
    tf.keras.layers.Softmax()
    ])
pre_model.summary()
pre_model.compile(
    loss='mse',
    optimizer='adam',
    metrics=[tf.keras.metrics.MeanSquaredError()],
)
pre_model.fit(pre_dataset, epochs=10)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten (Flatten)            (None, 26928)             0         
_________________________________________________________________
dropout (Dropout)            (None, 26928)             0

2022-05-06 07:07:29.503117: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2022-05-06 07:07:29.896221: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7


214/214 [==============================] - 12s 58ms/step - loss: 0.0015 - mean_squared_error: 0.0015
Epoch 2/10
  1/214 [..............................] - ETA: 11s - loss: 0.0014 - mean_squared_error: 0.0014

2022-05-06 07:07:41.309478: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-05-06 07:07:41.309585: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


214/214 [==============================] - 8s 38ms/step - loss: 0.0015 - mean_squared_error: 0.0015
Epoch 3/10
  3/214 [..............................] - ETA: 8s - loss: 0.0015 - mean_squared_error: 0.0015 

...

2022-05-06 07:08:37.017578: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2022-05-06 07:08:37.017653: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


214/214 [==============================] - 8s 37ms/step - loss: 0.0015 - mean_squared_error: 0.0015
Epoch 10/10
  3/214 [..............................] - ETA: 8s - loss: 0.0015 - mean_squared_error: 0.0015 

2022-05-06 07:08:44.938721: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
2022-05-06 07:08:44.938754: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


214/214 [==============================] - 8s 37ms/step - loss: 0.0015 - mean_squared_error: 0.0015


2022-05-06 07:08:52.858821: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
2022-05-06 07:08:52.858847: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


In [4]:
img_path = data_dir + '/image_data/ILSVRC/Data/CLS-LOC/train/'
filename = output_dir + 'image_data/' + sub + 'bold_recons.tfrecords'
writer_recons = tf.io.TFRecordWriter(filename)
categories = os.listdir(img_path)
for cat in categories[:2]:
    tmp = img_path + '/' + cat + '/'
    names = os.listdir(tmp)
    for name in names:
        image = tf.keras.preprocessing.image.load_img(tmp + name, target_size=(331, 331))
        image = tf.keras.preprocessing.image.img_to_array(image)
        x = tf.reshape(tf.cast(tf.image.resize(image, (int(img_rows), int(img_cols))),
                              tf.float32) / 127.5 - 1, (1, img_rows, img_cols, channels))
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = tf.keras.applications.nasnet.preprocess_input(image)  
        yhat = nasnet.predict(image)
        example = tf.train.Example(features=tf.train.Features(feature={
                'x': tf.train.Feature(float_list=tf.train.FloatList(value=yhat[0])),
                'y': tf.train.Feature(float_list=tf.train.FloatList(value=np.reshape(x, (-1))))
                }))
        writer_recons.write(example.SerializeToString())
writer_recons.close()

In [5]:
batch_size = 10
dataset = tf.data.TFRecordDataset(
    filenames = [output_dir + 'image_data/' + sub + 'bold_recons.tfrecords']).batch(batch_size)
batch_size = 10
def tf_parse(eg):
    example = tf.io.parse_example(
        eg, {
            'x': tf.io.FixedLenFeature(shape=(1000), dtype=tf.float32),
            'y': tf.io.FixedLenFeature(shape=(img_rows, img_cols, channels), dtype=tf.float32)
        })
    return example['x'], example['y']
dataset = dataset.map(tf_parse)

In [6]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(1000,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256)  # Note: None is the batch size
    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 256)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 28, 28, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, 
                                     activation='tanh'))
    assert model.output_shape == (None, 56, 56, 3)
    return model

model = make_generator_model()
model.summary()
r = model(np.random.rand(1,1000))
print(r.shape)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12544)             12544000  
_________________________________________________________________
batch_normalization (BatchNo (None, 12544)             50176     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 12544)             0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 7, 7, 256)         1638400   
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 256)         1024      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 256)        

In [7]:
model.compile(
    loss='mse',
    optimizer='adam',
    metrics=[tf.keras.metrics.MeanSquaredError()],
)
model.fit(dataset, epochs=4)

Epoch 1/4
260/260 [==============================] - 3s 10ms/step - loss: 0.3299 - mean_squared_error: 0.3299
Epoch 2/4
 28/260 [==>...........................] - ETA: 1s - loss: 0.3436 - mean_squared_error: 0.3436

2022-05-06 07:14:34.985327: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2022-05-06 07:14:34.985407: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


260/260 [==============================] - 1s 6ms/step - loss: 0.3225 - mean_squared_error: 0.3224
Epoch 3/4
 30/260 [==>...........................] - ETA: 1s - loss: 0.3370 - mean_squared_error: 0.3370

2022-05-06 07:14:36.473607: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2022-05-06 07:14:36.473689: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


260/260 [==============================] - 1s 6ms/step - loss: 0.3191 - mean_squared_error: 0.3190
Epoch 4/4
 29/260 [==>...........................] - ETA: 1s - loss: 0.3355 - mean_squared_error: 0.3355

2022-05-06 07:14:37.940270: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2022-05-06 07:14:37.940345: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


260/260 [==============================] - 1s 6ms/step - loss: 0.3158 - mean_squared_error: 0.3157


2022-05-06 07:14:39.406791: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2022-05-06 07:14:39.406873: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]
